This assumes that you already had .xml annotation files with one xml file per annotated frame given by rectlabel.  If you are using exported loopy annotations, you can use loppy_to_PASCAL.ipynb to convert the loopy annotations into the correct format.

In [1]:
import numpy as np
import os
import tensorflow as tf
from annotation_summary import get_annotation_summary
from create_label_map import create_label_map
from create_train_val import create_train_val
from create_tf_record import create_all_tf_records

In [2]:
#This is a folder with gazelle images and, within it, a folder called 'annotations' that has the .xml annotation files.
base_folder = '/home/golden/Projects_desktop/kenya-tracking/models/test'
image_folder = '/home/golden/demo_model_training/gazelle_images'

annotations_folder = os.path.join(image_folder, 'annotations')

# choose the 'pets' version of the model you are using here: 
# https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs
# one way to create the file is just copy and paste into a local text file
sample_config_file = '/home/golden/demo_model_training/faster_rcnn_resnet101_pets.config'


In [3]:
class_counts = get_annotation_summary(annotations_folder)

There are 336 .xml annotation files in the given folder.
{'head': 336}


In [4]:
#This is a list of the class names you want to use to clasiify the objects in the images.
#Generally they are the same as the classes that you annotated 
class_names = ['head']
class_names = list(class_counts.keys())
print(class_names)

['head']


In [5]:
#create nessisrary new folders
data_folder = os.path.join(base_folder, 'data')
model_folder = os.path.join(base_folder, 'model')
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
if not os.path.exists(model_folder):
    os.makedirs(model_folder)
    os.makedirs(os.path.join(base_folder, 'train'))
    os.makedirs(os.path.join(base_folder, 'eval'))


In [6]:
label_map_file = create_label_map(data_folder, class_names)

In [7]:
create_train_val(annotations_folder)

In [8]:
fraction_of_examples_for_evaluation_set = .3

create_all_tf_records(image_folder, label_map_file, data_folder,
                      fraction_of_examples_for_evaluation_set)

235 training and 102 validation examples.


/home/golden/tf-models/research/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


# Now you have to do stuff

## First download the model you want from the tf model zoo:

Go to https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
and download the model you want. This tutorial assumes 'faster_rcnn_resnet101_coco' but choose as you want. Copy and paste the contents of the resulting folder (but not the actual folder) into the folder written in the ouput of the cell bellow.

In [33]:
print('copy the contents of the downloaded model folder into the following folder:')
print(model_folder)

copy the contents of the downloaded model folder into the following folder:
/home/golden/Projects_desktop/kenya-tracking/models/v2_model/model


## Now you must edit the model .config file to match your use. 

Now you must edit the model pipeline.config file that you just downloaded and moved to match your use. The config file should be edited as described in the output of the following cell.

In [34]:
print('The .config file should be in this folder: ', model_folder)
print('The following edits should be made to the .config file:')
print()
print('Near the top of the file, the line that begins \'num_classes\' should be changed to...')
print('num_classes:', len(class_names))
print()
print('The line that begins \'fine_tune_checkpoint\' should be changed to...')
print('fine_tune_checkpoint: "' + os.path.join(model_folder, 'model.ckpt') + '"')
print()
print('Very important.  If you don\' include next line model will train forever.')
print('Beneath the line that says \'from_detection_checkpoint: true\' add the following line at the same indent as the above line (if running on gazelle head demo)')
print('num_steps: 5000')
print()
print('The line that begins \'label_map_path\' should be changed to...')
print('label_map_path: "' + os.path.join(data_folder, 'label_map.pbtxt') + '"')
print()
print('Within the braces ({}) labeled \'train_input_reader\' ')
print('The line that begins \'input_path\' should be changed to...')
print('input_path: "' + os.path.join(data_folder, 'train.record') + '"')
print()
print('The line that begins \'label_map_path\' should be changed to...')
print('label_map_path: "' + os.path.join(data_folder, 'label_map.pbtxt') + '"')
print()
print('Within the braces ({}) labeled \'eval_input_reader\' ')
print('The line that begins \'input_path\' should be changed to...')
print('input_path: "' + os.path.join(data_folder, 'val.record') + '"')
print()

The .config file should be in this folder:  /home/golden/Projects_desktop/kenya-tracking/models/v2_model/model
The following edits should be made to the .config file:

Near the top of the file, the line that begins 'num_classes' should be changed to...
num_classes: 5

The line that begins 'fine_tune_checkpoint' should be changed to...
fine_tune_checkpoint: "/home/golden/Projects_desktop/kenya-tracking/models/v2_model/model/model.ckpt"

Very important.  If you don' include next line model will train forever.
Beneath the line that says 'from_detection_checkpoint: true' add the following line at the same indent as the above line (if running on gazelle head demo)
num_steps: 5000

The line that begins 'label_map_path' should be changed to...
label_map_path: "/home/golden/Projects_desktop/kenya-tracking/models/v2_model/data/label_map.pbtxt"

Within the braces ({}) labeled 'train_input_reader' 
The line that begins 'input_path' should be changed to...
input_path: "/home/golden/Projects_deskto

Now you are ready to train the model

In [4]:
def run_training(train_py, eval_py, pipeline_config, train_dir, eval_dir, gpu_count=1, name=None):  
    print('running ', name)
    %env CUDA_VISIBLE_DEVICES=0
    train_process = subprocess.Popen(['python', train_py,
                            '--logtostderr', 
                            '--pipeline_config_path=' + pipeline_config,
                            '--train_dir=' + train_dir],
                            preexec_fn=os.setsid)
    if gpu_count > 1:

        time.sleep(60)

        %env CUDA_VISIBLE_DEVICES=1
        eval_process = subprocess.Popen(['python', eval_py,
                                '--logtostderr', 
                                '--pipeline_config_path=' + pipeline_config,
                                '--checkpoint_dir=' + train_dir,
                                '--eval_dir=' + eval_dir],
                                preexec_fn=os.setsid)

    train_process.wait()
    
    if gpu_count > 1:

        print('sleeping')

        time.sleep(180)

        print('shutting down eval')
        
        print('killing eval...')

        os.killpg(os.getpgid(eval_process.pid), signal.SIGTERM)
    
    time.sleep(30)

In [5]:
import subprocess
import signal
import time

#these must be changed to match where these files are saved on your computer
train_py = '/home/golden/tf-models/research/object_detection/train.py'   
eval_py = '/home/golden/tf-models/research/object_detection/eval.py'    

pipeline_config = os.path.join(model_folder, 'pipeline.config')
train_dir = os.path.join(base_folder, 'train') 
eval_dir = os.path.join(base_folder, 'eval')


In [6]:
#only runs eval if two GPUs
gpu_count = 2

run_training(train_py, eval_py, pipeline_config, train_dir, eval_dir, gpu_count, name='')

running  
env: CUDA_VISIBLE_DEVICES=0
env: CUDA_VISIBLE_DEVICES=1
sleeping
shutting down eval
killing eval...


You can monitor training by running the following output in the terminal: 

In [ ]:
print('tensorboard --logdir="' + base_folder + '"')

# Export Graph

In [7]:
#these must be changed to match where these files are saved on your computer
export_py = '/home/golden/tf-models/research/object_detection/export_inference_graph.py'
#choose step number to export (in train folder)
step_num = 120000

subprocess.Popen(['python', export_py,
                '--input_type image_tensor', 
                '--pipeline_config_path=' + pipeline_config,
                '--trained_checkpoint_prefix=' + os.path.join(train_dir, 'model.ckpt-' + str(step_num)),
                '--output_directory=' + os.path.join(base_folder, 'out_put_inference_graph.pb')],
                preexec_fn=os.setsid)

